교재 p434 NLP Electra

In [1]:
import json
import numpy as np
import pandas as pd
import torch

file_path = './mecab/ner_all.json'

In [2]:
# JSON 파일 열기
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

location = []

# 데이터 구조확인
for a in data:
    print(f"토큰: { a['tokens']}")
    print(f"위치 데이터: { a['ner_tags']}")
    location.append(a['ner_tags'])

토큰: ['명태', '본가', '에서', '명란젓', '비빔밥', '두', '개', '랑', '된장찌개', '하나', '추가', '요', '.']
위치 데이터: [1, 2, 0, 3, 4, 5, 6, 0, 3, 5, 0, 0, 0]
토큰: ['우상', '화로구이', '에서', '갈비', '한', '판', ',', '된장찌개', '하나', ',', '공기', '밥', '두', '개', '부탁', '해요', '.']
위치 데이터: [1, 2, 0, 3, 5, 6, 0, 3, 5, 0, 3, 4, 5, 6, 0, 0, 0]
토큰: ['오늘', '은', '볶음밥', '에서', '김치볶음밥', '두', '개', '랑', '계란', '후', '라', '이', '추가', '요', '.']
위치 데이터: [1, 2, 2, 0, 3, 5, 6, 0, 3, 4, 4, 4, 0, 0, 0]
토큰: ['배달', '비', '무료', '삼', '겹', '배무', '삼', '에서', '삼겹살', '김치찌개', '하나', '랑', '공기', '밥', '두', '개', '시킬', '게', '요', '.']
위치 데이터: [1, 2, 2, 2, 2, 2, 2, 0, 3, 4, 5, 0, 3, 4, 5, 6, 0, 0, 0, 0]
토큰: ['국밥', '상회', '&', '냉면', '에서', '돼지국밥', '두', '개', ',', '물냉면', '하나', '주문', '할게요', '.']
위치 데이터: [1, 2, 2, 2, 0, 3, 5, 5, 0, 3, 5, 0, 0, 0]
토큰: ['또', '근', '이', '찜', '닭', '에서', '닭찜', '한', '마리', '랑', '감자튀김', '하나', '추가', '요', '.']
위치 데이터: [1, 2, 2, 2, 2, 0, 3, 5, 6, 0, 3, 5, 0, 0, 0]
토큰: ['본가', '낙', '곱새', '에서', '낙', '곱새', '두', '개', '랑', '곤약', '면', '하나', '추가', '해', '주', '세요', '.'

In [3]:
# 혹시나 패딩할 일이 생기면 max값을 알기위해 > 이거 함수로 같이 넣어넣으면 좋을듯
max_len = max(location, key=len)
len(max_len)

25

In [4]:
# 2. 토크나이저 및 데이터 전처리

import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer

# Electra 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")

# 데이터를 처리하는 함수
def preprocess_data(example):
    tokens = example["tokens"]
    ner_tags = example["ner_tags"]

    # 토크나이저를 사용해 토큰화
    tokenized_inputs = tokenizer(
        tokens,
        is_split_into_words=True,  # 토큰 리스트를 입력으로 사용
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # 토큰화된 결과를 원래 토큰에 매핑
    word_ids = tokenized_inputs.word_ids()
    labels = []

    for word_id in word_ids:
        if word_id is None:  # 특수 토큰 ([CLS], [SEP] 등)
            labels.append(-100)
        else:  # 해당 word_id에 대응하는 원래 ner_tag 추가
            labels.append(ner_tags[word_id])

    return tokenized_inputs, labels

c:\Users\KDP-23\anaconda3\envs\imc_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# NER 데이터 배치를 패딩하여 텐서로 반환.

def collate_fn(batch):
   
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]

    # 입력 데이터와 레이블 모두 패딩 처리
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # 패딩 시 레이블 -100으로 설정

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
# 모든 데이터를 처리
processed_data = [preprocess_data(example) for example in data]

In [7]:
for inputs, labels in processed_data:
    print("Input IDs:", inputs["input_ids"])
    print("Attention Mask:", inputs["attention_mask"])
    print("Labels:", labels)

Input IDs: tensor([[    2, 28419,  2783,  4070,  6215,  2675,  4271,  5018, 20475,  2419,
          2027,  2531, 28026,  6272,  6565,  3183,    18,     3]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Labels: [-100, 1, 2, 2, 0, 3, 3, 3, 4, 5, 6, 0, 3, 5, 0, 0, 0, -100]
Input IDs: tensor([[    2, 14759, 27339, 23361,  6215, 13274,  3757,  3681,    16, 28026,
          6272,    16,  8103,  2739,  2419,  2027,  8161,  7489,    18,     3]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Labels: [-100, 1, 2, 2, 0, 3, 5, 6, 0, 3, 5, 0, 3, 4, 5, 6, 0, 0, 0, -100]
Input IDs: tensor([[    2,  6451,  3231, 20093,  6215,  9370, 24798,  4717,  2419,  2027,
          2531, 11581,  3826,  2523,  3240,  6565,  3183,    18,     3]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Labels: [-100, 1, 2, 2, 0, 3, 3, 3, 5, 6, 0, 3, 4, 4, 4, 0, 0, 0, -100]
Input IDs: tensor([[    2, 10222,  2818,

In [8]:
from torch.utils.data import DataLoader, Dataset

# PyTorch Dataset 정의
class NERDataset(Dataset):
    def __init__(self, processed_data):
        self.processed_data = processed_data

    def __len__(self):
        return len(self.processed_data)

    def __getitem__(self, idx):
        inputs, labels = self.processed_data[idx]
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels": torch.tensor(labels)
        }

# Dataset 생성
dataset = NERDataset(processed_data)

# DataLoader 생성
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)


In [ ]:
from transformers import AutoModelForTokenClassification, AutoModelForSequenceClassification, TextClassificationPipeline
from sklearn.metrics import precision_recall_fscore_support


# model = AutoModelForSequenceClassification.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
# sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)


# NER 모델 로드
# model = AutoModelForTokenClassification.from_pretrained(
#     "google/electra-small-discriminator",
#     num_labels=7  # NER 태그 클래스 개수
# )
model = AutoModelForSequenceClassification.from_pretrained(
    "jaehyeong/koelectra-base-v3-generalized-sentiment-analysis",
    num_labels=7)


# 예측 및 평가지표 계산
for batch in dataloader:
    # 모델에 입력
    outputs = model(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        labels=batch["labels"]
    )

    # 손실 값
    loss = outputs.loss
    print("Loss:", loss.item())

    # 로짓(logits) -> 예측 태그
    predictions = torch.argmax(outputs.logits, dim=-1)

    # 레이블 및 패딩 제거 (-100 필터링)
    true_labels = batch["labels"].view(-1)
    pred_labels = predictions.view(-1)

    valid_indices = true_labels != -100
    true_labels = true_labels[valid_indices]
    pred_labels = pred_labels[valid_indices]

    # 평가지표 계산
    precision, recall, f1, _ = precision_recall_fscore_support(
        true_labels.cpu().numpy(),
        pred_labels.cpu().numpy(),
        average="macro"  # 다중 클래스에 대해 평균 계산
    )

    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")


c:\Users\KDP-23\anaconda3\envs\imc_env\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\KDP-23\.cache\huggingface\hub\models--jaehyeong--koelectra-base-v3-generalized-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: Error(s) in loading state_dict for ElectraForSequenceClassification:
	size mismatch for classifier.out_proj.weight: copying a param with shape torch.Size([2, 768]) from checkpoint, the shape in current model is torch.Size([7, 768]).
	size mismatch for classifier.out_proj.bias: copying a param with shape torch.Size([2]) from checkpoint, the shape in current model is torch.Size([7]).
	You may consider adding `ignore_mismatched_sizes=True` in the model `from_pretrained` method.